In [1]:
import tweepy
import keyring
import datetime
import pandas as pd
import numpy as np

## Parameters

In [2]:
days_to_post = 30
# only works for 1 or 2 posts per day, since only the date of the most recent post is checked in
# chapter 4
posts_per_day = 2
pics_per_day = 2
texts_per_day = 1
#starting_date = datetime.date(2022, 5, 9)
starting_date = datetime.date.today()
user_id_dog = 1483397937787023362
user_id_cat = 0

## Create data structure

In [3]:
# create list with filenames for the dog pictures (1-60.jpg)
dogpics = list(range(days_to_post*pics_per_day))
for num in dogpics:
    dogpics[num] = '../pics/dogs/'+str(num+1)+'.jpg'

In [4]:
# create list with filenames for the cat pictures (1-60.jpg)
catpics = list(range(days_to_post*pics_per_day))
for num in catpics:
    catpics[num] = '../pics/cats/'+str(num+1)+'.jpg'

In [5]:
# create list with dates to be posted
# testing was only done with consecutive dates, an error may occur if the date this script is run 
# is missing in the list
date_list = [starting_date + datetime.timedelta(days=x) for x in range(days_to_post)]
date_list = list(np.repeat(date_list, posts_per_day))

In [6]:
# create tweet texts for testing
testtext = list(range(60))
for num in testtext:
    testtext[num] = num+1

In [7]:
# create dataframe that assigns picture-filenames and tweet-texts to dates
dogs = pd.DataFrame(dogpics, columns=['pics'])
# this part will be replaced by actual tweet-texts from csv/excel file
##########################################
dogs['text'] = testtext
dogs['text'] = dogs.text.apply(lambda row: None if row%2!=0 else row)
dogs['text'] = dogs.text.apply(lambda row: None if np.isnan(row) else row/2)
dogs['text'] = dogs.text.apply(lambda row: None if np.isnan(row) else 'this is text number '+ str(int(row)))
##########################################
dogs['date'] = date_list

In [8]:
# create column that assigns the number 1 or 2 to the tweets of every day
# this represents the post order on that date
l = [[range(1,posts_per_day+1)]]
l = np.repeat(l, days_to_post, axis=1)
l = [item for sublist1 in l for sublist2 in sublist1 for item in sublist2]
dogs['post_of_day'] = l

## Twitter Authentification

In [9]:
client = tweepy.Client(keyring.get_password('twitter','bearer_token'), keyring.get_password('twitter','api_key'), keyring.get_password('twitter','api_key_secret'), keyring.get_password('twitter','access_token'), keyring.get_password('twitter','access_token_secret'))        

In [10]:
auth = tweepy.OAuthHandler(keyring.get_password('twitter','api_key'), keyring.get_password('twitter','api_key_secret'))
auth.set_access_token(keyring.get_password('twitter','access_token'), keyring.get_password('twitter','access_token_secret'))
api = tweepy.API(auth)

## Check if this day has already been posted

In [11]:
check = api.user_timeline(user_id=user_id_dog, count=1)

In [12]:
if check[0].created_at.date() != datetime.date.today():
    post_of_day = 1
else:
    post_of_day = 2

## Post

In [20]:
dog_post = dogs.loc[(dogs.date == datetime.date.today()) & (dogs.post_of_day == post_of_day)]
dog_text = dog_post.text.item()
#client.create_tweet(text = dog_text)

In [14]:
dog_pic = dogs.loc[(dogs.date == datetime.date.today()) & (dogs.post_of_day == post_of_day)]['pics'].item()
media_dog = api.media_upload(dog_pic)
client.create_tweet(text=dog_text, media_ids=[media_dog.media_id])

Response(data={'id': '1514237341845626887', 'text': 'this is text number 1 https://t.co/KiHn0vfxy0'}, includes={}, errors=[], meta={})